## Exploring experiments

In [ ]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = ''

from os import path
import logging
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from trojan_defender import (experiment, set_root_folder,
                             datasets, set_db_conf, plot,
                             get_db_conf)
from trojan_defender.detect import saliency_ as saliency
from trojan_defender import datasets
from pymongo import MongoClient

In [ ]:
# config logging
logging.basicConfig(level=logging.ERROR)

# matplotlib size
plt.rcParams['figure.figsize'] = (10, 10)

# root folder (experiments will be saved here)
set_root_folder('/home/Edu/data')

dump_folder = '/home/Edu/saliency'

# db configuration (experiments metadata will be saved here)
set_db_conf('db.yaml')

In [ ]:
clean = datasets.mnist()

def detect(directory):
    model, dataset, metadata = experiment.load(directory)
    try:
        score = saliency.score(model, clean, 500)
    except:
        print('failed...')
        return None
    return score

In [ ]:
def evaluate_experiment(experiment, dump=False):
    # directory = model['directory']
    experiment = list(experiment)
    mean_acc = np.mean([model['metrics']['accuracy_score'] for model in experiment])
    scores = [detect(model['directory']) for model in experiment]
    return mean_acc, scores

In [ ]:
conf = get_db_conf()
client = MongoClient(conf['uri'])
con = client[conf['db']][conf['collection']]

In [ ]:
# baseline
baseline = con.find({'dataset.name': 'MNIST',
                     'group_name': 'prezi',
                     'metrics.patch_success_rate': {"$exists": False}})

# attacks
# 'metrics.patch_success_rate': {"$lt": 0.05},
attacked = con.find({'dataset.name': 'MNIST',
                     'group_name': 'prezi',
                     'dataset.dynamic_mask': False,
                     'dataset.dynamic_pattern': False,
                     'dataset.fraction': 0.15,
                     'dataset.type_': 'sparse',
                     'dataset.proportion': 0.01})

In [ ]:
for b in baseline:
    print(b)

In [ ]:
attacked = list(attacked)

In [ ]:
attacked[0]

In [ ]:
np.mean([a['metrics']['patch_success_rate'] for a in attacked])

In [ ]:
np.mean([a['metrics']['accuracy_score'] for a in attacked])

In [ ]:
res = evaluate_experiment(attacked)

In [ ]:
print('mean acc:', res[0])

In [ ]:
scores = [r for r in res[1] if r is not None]
np.mean([r[1] > 0.5 for r in scores])

In [ ]:
scores

In [ ]:
res_baseline = [detection_round(model) for model in baseline]

In [ ]:
res_noisy = [detection_round(model) for model in noisy]

## Plots

In [ ]:
success = [r[1] for r in res]
mode_changes = [r[3] for r in res]

plt.scatter(success, mode_changes, alpha=0.5)